In [1]:
import numpy as np
import pandas as pd

In [2]:
np.random.seed(42)
data=pd.read_csv('fraudTest.csv')
data.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,...,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,3,2020-06-21 12:15:15,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,...,28.5697,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,4,2020-06-21 12:15:17,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,...,44.2529,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0


In [3]:
df=data[['category','amt','gender','city','state','lat','long','job','is_fraud']]
df.head()

,category,amt,gender,city,state,lat,long,job,is_fraud
0,personal_care,2.86,M,Columbia,SC,33.9659,-80.9355,Mechanical engineer,0
1,personal_care,29.84,F,Altonah,UT,40.3207,-110.4360,"Sales professional, IT",0
2,health_fitness,41.28,F,Bellmore,NY,40.6729,-73.5365,"Librarian, public",0
3,misc_pos,60.05,M,Titusville,FL,28.5697,-80.8191,Set designer,0
4,travel,3.19,M,Falmouth,MI,44.2529,-85.0170,Furniture designer,0


In [4]:
from sklearn.utils import resample
df_majority=df[df.is_fraud==0]
df_minority=df[df.is_fraud==1]
print(df_majority.shape, df_minority.shape)

(553574, 9) (2145, 9)


In [5]:
df_minority_unsampled=resample(df_minority,n_samples=100000,random_state=42)
mydata2=pd.concat([df_majority[:100000],df_minority_unsampled])
print(df_minority_unsampled.shape,mydata2.shape)

(100000, 9) (200000, 9)


In [6]:
mydata2['is_fraud'].value_counts()

0    100000
1    100000
Name: is_fraud, dtype: int64

In [7]:
from sklearn.preprocessing import LabelEncoder

l1=LabelEncoder()
cut_l =l1.fit(mydata2["category"])
mydata2["category_l"]=l1.transform(mydata2["category"])

l2=LabelEncoder()
gen_l=l2.fit(mydata2["gender"])
mydata2["gender_l"]=l2.transform(mydata2['gender'])

l3=LabelEncoder()
city_l=l3.fit(mydata2["city"])
mydata2["city_l"]=l3.transform(mydata2['city'])

l4=LabelEncoder()
state_l=l4.fit(mydata2["state"])
mydata2["state_l"]=l4.transform(mydata2["state"])

l5=LabelEncoder()
job_l=l5.fit(mydata2["job"])
mydata2["job_l"]=l5.transform(mydata2["job"])

In [8]:
df=mydata2.drop(['category','gender','city','state','job'],axis=1)
df.head()

,amt,lat,long,is_fraud,category_l,gender_l,city_l,state_l,job_l
0,2.86,33.9659,-80.9355,0,10,1,157,39,275
1,29.84,40.3207,-110.4360,0,10,0,16,43,392
2,41.28,40.6729,-73.5365,0,5,0,61,33,259
3,60.05,28.5697,-80.8191,0,9,1,764,8,407
4,3.19,44.2529,-85.0170,0,13,1,247,21,196


In [9]:
cat_list=list(df["category_l"].unique())
cat_list

[10, 5, 9, 13, 7, 12, 1, 6, 0, 11, 8, 4, 2, 3]

In [10]:
y=df['is_fraud'] #Dependent Variable
x= df.drop('is_fraud',axis=1) 

from sklearn import preprocessing
x=preprocessing.normalize(x)

In [11]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.15, random_state=42)

In [12]:
from sklearn.ensemble import RandomForestClassifier

clf=RandomForestClassifier(n_estimators=100)
clf.fit(x_train,y_train)
y_pred=clf.predict(x_test)

In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print("Accuracy score: {}". format(accuracy_score(y_test, y_pred)) )
print("Precision score: {}". format(precision_score(y_test, y_pred)) )
print("Recall score: {}". format(recall_score(y_test, y_pred)))
print("F1 score: {}". format(f1_score(y_test, y_pred)))

Accuracy score: 0.9977
Precision score: 0.9954506494362761
Recall score: 1.0
F1 score: 0.9977201387741615
